In [114]:
import pandas
import numpy as np
import sklearn

In [99]:
data = pandas.read_csv('deltasSubstraction.csv',sep=";|,", engine="python")

In [100]:
data = data.drop(['REPORT_MONTH','CLIENT_HASH_ID','GENDER','F1','AGE'],1)

In [101]:
active = data.loc[data['ACTIVE_FLAG']==1].sample(n=22031,random_state = 1)
nonactive = data.loc[data['ACTIVE_FLAG']==0]
dataset = pandas.concat([active,nonactive])

In [102]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier()
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=100)

In [104]:
train = dataset.sample(n=22031,random_state = 7)
features = train.drop(['ACTIVE_FLAG'],1)
goals = train['ACTIVE_FLAG']
predict = dataset.sample(n=22031,random_state = 191)

In [117]:
#train = data
#features = train.drop(['ACTIVE_FLAG'],1)
#goals = train['ACTIVE_FLAG']
#predict = data

In [118]:
clf.fit(features, goals)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [119]:
truval = predict['ACTIVE_FLAG'].values
res = clf.predict(predict.drop(['ACTIVE_FLAG'],1))
tp = 0
fp = 0
tn = 0
fn = 0
for goal, predicted in zip(truval,res):
    if bool(goal):
        if predicted == 1:           
            tp += 1
            continue
        fn += 1
        continue
    if bool(predicted):
        fp += 1
        continue
    tn += 1

        
print(tp)
print(tn)
print(fp)
print(fn)
print("Полнота уходящих")
print(float(tn)/(tn+fp))

print("Полнота оставшихся")
print(float(tp)/(tp+fn))

print("Accuracy")
print(float(tp+tn)/(tp+fp+tn+fn))

prec = float(tp)/(tp+fp)
recall = float(tp)/(tp+fn)
print("Precision")
print(prec)
print("Recall")
print(recall)
print("Fmeasure")
print (2*prec*recall/(prec+recall))

574182
16311
5720
162
Полнота уходящих
0.740365848123
Полнота оставшихся
0.999717939075
Accuracy
0.990137078181
Precision
0.990136264403
Recall
0.999717939075
Fmeasure
0.994904032589


In [120]:
predict.ACTIVE_FLAG.value_counts()

1    574344
0     22031
Name: ACTIVE_FLAG, dtype: int64

In [121]:
zip(list(dataset)[1:],clf.feature_importances_)

[('TRN_CNT', 0.068432749483244959),
 ('TRN_CNT_POS', 0.060304934581320593),
 ('TRN_CNT_ATM', 0.047064273570761796),
 ('TRN_AMNT', 0.18090073925781394),
 ('TRN_AMNT_POS', 0.12594163949163006),
 ('TRN_AMNT_ATM', 0.15991080911191227),
 ('CONTRACT_TURN_AMOUNT', 0.17844377060845448),
 ('SALARY_INCOME', 0.040968206800416629),
 ('DBO_AUTH_WWW', 0.033868940003132737),
 ('DBO_AUTH_MB', 0.025710017330682438),
 ('DBO_OPER_CNT', 0.012450312287105879),
 ('DBO_REM_CNT', 0.0068308713503501791),
 ('DBO_PAY_CNT', 0.011149505966238331),
 ('DBO_OPER_AMNT', 0.02073702658829887),
 ('DBO_REM_AMNT', 0.012241299132517399),
 ('DBO_PAY_AMNT', 0.015044904436119486)]